In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

# Internal packages
from data_loader import KidneyStoneDataset, ToTensor
from model import Kidney_net, neg_loglik
from train import train

In [2]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 3

# Initialize the dataset
data = KidneyStoneDataset("./data/kidney_data.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = Kidney_net(N_HU)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [3]:
cum_loss = train(model, optimizer, neg_loglik, train_loader, EPOCHS)

In [4]:
# We want to query an intervention on the treatment. In order to do that, we estimate p(R=1|L=1, T=do(A))*P(L=1)
l_1_t_1_r_1 = torch.tensor([1., 1., 1.])
l_0_t_1_r_1 = torch.tensor([0., 1., 1.])

# We want to query an intervention on the treatment. In order to do that, we estimate p(R=1|L=1, T=do(B))*P(L=1)
l_1_t_0_r_1 = torch.tensor([1., 0., 1.])
l_0_t_0_r_1 = torch.tensor([0., 0., 1.])

In [5]:
# Probabilities with Treatment A
p_ks, p_t1_l1, p_r1_t1_l1 = model(l_1_t_1_r_1.unsqueeze(0)) # Probabilities with KS = L
_, p_t1_l0, p_r1_t1_l0 = model(l_0_t_1_r_1.unsqueeze(0))    # Probabilities with KS = S

# Probabilities with Treatment B
_, p_t0_l1, p_r1_t0_l1 = model(l_1_t_0_r_1.unsqueeze(0))  # Probabilities with KS = L
_, p_t0_l0, p_r1_t0_l0 = model(l_0_t_0_r_1.unsqueeze(0))  # Probabilities with KS = S

In [17]:
print("The estimated probability of a Large kidney stone is: %.4f\n\
The estimated probability of Recovery given large stones and treatment A is: %.4f\n\
The estimated probability of Recovery given large stones and treatment B is: %.4f\n\
The estimated probability of Recovery given small stones and treatment A is: %.4f\n\
The estimated probability of Recovery given large stones and treatment B is: %.4f\n" \
      % (p_ks, p_r1_t1_l1, p_r1_t0_l1, p_r1_t1_l0, p_r1_t0_l0))

The estimated probability of a Large kidney stone is: 0.4897
The estimated probability of Recovery given large stones and treatment A is: 0.7558
The estimated probability of Recovery given large stones and treatment B is: 0.6968
The estimated probability of Recovery given small stones and treatment A is: 0.9568
The estimated probability of Recovery given large stones and treatment B is: 0.8894



In [18]:
int_t1 = p_r1_t1_l1*p_ks + p_r1_t1_l0*(1-p_ks)
int_t0 = p_r1_t0_l1*p_ks + p_r1_t0_l0*(1-p_ks)

c_effect = int_t1 - int_t0
print(c_effect.item())

0.06325888633728027


In [51]:
an_t1 = (192/263)*343/(343+357) + (81/87)*(357/(343+357))
an_t0 = (55/80)*343/(343+357) + (234/270)*(357/(343+357))
print("The analytical solution for P(R=1 | do(T=A))is: %.4f and the estimated value is %.4f " % (an_t1, int_t1.item()))
print("The analytical solution for P(R=1 | do(T=B))is: %.4f and the estimated value is %.4f " % (an_t2, int_t0.item()))
print("The analytical causal effect P(R=1 | do(T=A)) - P(R=1 | do(T=B)) is: %.4f and the estimated value is %.4f " % (an_t1-an_t0, int_t1.item()-int_t0.item()))

The analytical solution for P(R=1 | do(T=A))is: 0.8325 and the estimated value is 0.8018 
The analytical solution for P(R=1 | do(T=B))is: 0.7789 and the estimated value is 0.7808 
The analytical causal effect P(R=1 | do(T=A)) - P(R=1 | do(T=B)) is: 0.0537 and the estimated value is 0.0210 
